## TicTacToe

- X와 O가 번갈아가며 두는 삼목(오목의 3x3짜리 버전) 게임 문제
    - 현재 플레이어가 누구인지 명시하고, 플레이어를 교체할 수 있어야 함
    - 현재 상태에서 어떤 수를 둘 수 있는지 확인하고, 수를 둘 수 있어야 함
    - 수를 둔 후에 새로운 보드 상태를 반환해야 함
    - 게임의 종료 상태를 판단 및 특정 플레이어의 승리 여부 확인할 수 있어야 함
    - 종료 상태 도달 시 효용 값 반환할 수 있어야 함

In [34]:
class TicTacToe:
    def __init__(self):
        self.board = [' '] * 9  # 9개의 빈 칸으로 구성된 보드 생성
        self.player = 'X'  # 첫 번째 플레이어를 'X'로 설정
        self.round = 1 # 게임 턴

    def empty_cells(self): # 가능한 수 추적 (현재 보드에서 빈 칸의 인덱스 반환)
        return [i for i, cell in enumerate(self.board) if cell == ' ']

    def valid_move(self, x): # 주어진 위치 x 에 수를 둘 수 있는지 판단
        return x in self.empty_cells()

    def move(self, x): # 플레이어가 위치 x에 수를 놓음
        if self.valid_move(x):
            self.board[x] = self.player
            return True
        return False

    def check_win(self, player): # 특정 플레이어가 승리했는지 확인
        win_conditions = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # 가로 승리 조건
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # 세로 승리 조건
            [0, 4, 8], [2, 4, 6]              # 대각선 승리 조건
        ]
        for condition in win_conditions:
            if all(self.board[i] == player for i in condition): # 인자로 받은 요소가 모두 True일때 True 반환, 하나라도 False가 있으면 False 반환, 즉 위에 조건 중 [x, y, z] 모두가 해당 플레이어일때 실행
                return True 
        return False

    def is_terminal(self): # 게임이 종료되었는지 확인 (승리, 패배, 무승부-승자없고 빈칸없을때)
        return self.check_win('X') or self.check_win('O') or not self.empty_cells()

    def utility(self): # 게임의 결과에 따른 점수 반환
        if self.check_win('X'): # 승리
            return 1
        elif self.check_win('O'): # 패배
            return -1
        else: # 무승부
            return 0  

    def switch_player(self): # 현재 수를 둘 플레이어 변경
        self.player = 'O' if self.player == 'X' else 'X'

    def result(self, move): # 보드에서 특정 수를 두었을 때 결과 보드를 반환 (새로운 상태)
        new_game = TicTacToe()
        new_game.board = self.board[:] 
        new_game.player = self.player
        new_game.round = self.round
        new_game.move(move)
        new_game.switch_player()
        return new_game

    def __str__(self):
        board_str = f'== Round {self.round} ==\n'
        for i, cell in enumerate(self.board):
            if i % 3 == 0 and i != 0:
                board_str += '\n'
            board_str += f'|{cell}|'
        return board_str

## MINIMAX 알고리즘

### 의사코드

- 이 알고리즘의 핵심은 MAX-VALUE 및 MIN-VALUE 함수 에서 MAX 및 MIN 플레이어가 최선의 행동을 택하는데, MAX는 가장 높은 값을 찾고, MIN은 반대로 가장 낮은 값을 찾도록 설계하는 것임

```java

Function MINIMAX(game, state)
  player ← game.TO_MOVE(state)
  value, move ← MAX_VALUE(game, state)
  Return move

Function MAX_VALUE (game, state)
  if game.IS-TERMINAL(state) then return game.UTILITY(state, player), null
  v, move ← -∞, null
  for each a in game.ACTIONS(state) do
    v2, a2 ← MIN_VALUE(game, game.RESULT(state, a))
    if v2 > v then
      v, move ← v2, a
  return v, move

Function MIN_VALUE (game, state)
  if game.IS-TERMINAL(state) then return game.UTILITY(state, player), null
  v, move ← +∞, null
  for each a in game.ACTIONS(state) do
    v2, a2 ← MAX_VALUE(game, game.RESULT(state, a))
    if v2 < v then
      v, move ← v2, a
  return v, move

```

In [39]:
def minimax(game):
    if game.is_terminal():
        if game.check_win('X'):
            return 'X 승리!'
        elif game.check_win('O'):
            return 'O 승리!'
        else:
            return '비겼습니다!'
    else:
        if game.player == 'X':  # 플레이어 X로 시작하면 최대값 탐색으로 시작
            _, move = max_value(game)
        else:  # 플레이어 O로 시작하면 최소값 탐색으로 시작
            _, move = min_value(game)
        game.move(move)
        game.switch_player() # 수를 두고 플레이어를 교체
        print(game)
        game.round += 1
        return minimax(game)  # 자기 자신을 재귀적으로 호출

def max_value(game): # X 관점에서 가능한 행동에 대하여 가장 큰 최소최대값을 반환
    if game.is_terminal():
        return game.utility(), None
    value, move = -float('inf'), None
    for action in game.empty_cells():
        next_game = game.result(action)
        v, _ = min_value(next_game)
        if v > value:
            value, move = v, action
    return value, move

def min_value(game): # O 관점에서 가능한 행동에 대하여 가장 낮은 최소최대값을 반환
    if game.is_terminal():
        return game.utility(), None
    value, move = float('inf'), None
    for action in game.empty_cells():
        next_game = game.result(action)
        v, _ = max_value(next_game)
        if v < value:
            value, move = v, action
    return value, move

In [40]:
# 게임 실행
game = TicTacToe()
result = minimax(game)
print(result)

== Round 1 ==
|X|| || |
| || || |
| || || |
== Round 2 ==
|X|| || |
| ||O|| |
| || || |
== Round 3 ==
|X||X|| |
| ||O|| |
| || || |
== Round 4 ==
|X||X||O|
| ||O|| |
| || || |
== Round 5 ==
|X||X||O|
| ||O|| |
|X|| || |
== Round 6 ==
|X||X||O|
|O||O|| |
|X|| || |
== Round 7 ==
|X||X||O|
|O||O||X|
|X|| || |
== Round 8 ==
|X||X||O|
|O||O||X|
|X||O|| |
== Round 9 ==
|X||X||O|
|O||O||X|
|X||O||X|
비겼습니다!


```java
Function ALPHABETA(game, state)
  player ← game.TO-MOVE(state)
  value, move ← MAX_VALUE(game, state, -∞, +∞)
  return move

Function MAX_VALUE (game, state, α, β)
  if game.IS-TERMINAL(state) then return game.UTILITY(state, player), null
  v, move ← -∞, null
  for each a in game.ACTIONS(state) do
      v2, a2 ← MIN_VALUE(game, game.RESULT(state, a), α, β)
      if v2 > v then
          v, move ← v2, a
          α ← MAX(α, v)
      if v >= β then break
  return v, move

Function MIN_VALUE (game, state , α, β)
  if game.IS-TERMINAL(state) then return game.UTILITY(state, player), null
  v, move ← +∞, null
  for each a in game.ACTIONS(state) do
      v2, a2 ← MAX_VALUE(game, game.RESULT(state, a), α, β)
      if v2 < v then
          v, move ← v2, a
          β ← MIN(β, v)
      if v <= α then break
  return v, move
```

In [41]:
def alphabeta(game):
    if game.is_terminal():
        if game.check_win('X'):
            return 'X 승리!'
        elif game.check_win('O'):
            return 'O 승리!'
        else:
            return '비겼습니다!'
    else:
        if game.player == 'X':
            _, move = max_value_ab(game, -float('inf'), float('inf'))
        else:
            _, move = min_value_ab(game, -float('inf'), float('inf'))
        game.move(move)
        print(game)
        game.switch_player()
        game.round += 1
        return alphabeta(game)

def max_value_ab(game, alpha, beta):
    if game.is_terminal():
        return game.utility(), None
    value, move = -float('inf'), None
    for action in game.empty_cells():
        next_game = game.result(action)
        next_value, _ = min_value_ab(next_game, alpha, beta)
        if next_value > value:
            value, move = next_value, action
        alpha = max(alpha, value)
        if value >= beta:
            break  # 가지치기
    return value, move

def min_value_ab(game, alpha, beta):
    if game.is_terminal():
        return game.utility(), None
    value, move = float('inf'), None
    for action in game.empty_cells():
        next_game = game.result(action)
        next_value, _ = max_value_ab(next_game, alpha, beta)
        if next_value < value:
            value, move = next_value, action
        beta = min(beta, value)
        if value <= alpha:
            break  # 가지치기
    return value, move

In [42]:
game = TicTacToe()
result = alphabeta(game)
print(result) # 항상 완벽하게 같은 결과만 나옴 왜냐면 이 알고리즘 자체가 결정론적(deterministic) 알고리즘이기 때문

== Round 1 ==
|X|| || |
| || || |
| || || |
== Round 2 ==
|X|| || |
| ||O|| |
| || || |
== Round 3 ==
|X||X|| |
| ||O|| |
| || || |
== Round 4 ==
|X||X||O|
| ||O|| |
| || || |
== Round 5 ==
|X||X||O|
| ||O|| |
|X|| || |
== Round 6 ==
|X||X||O|
|O||O|| |
|X|| || |
== Round 7 ==
|X||X||O|
|O||O||X|
|X|| || |
== Round 8 ==
|X||X||O|
|O||O||X|
|X||O|| |
== Round 9 ==
|X||X||O|
|O||O||X|
|X||O||X|
비겼습니다!
